In [7]:
'''
  code by Minho Ryu @bzantium
  
'''
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

from torch import FloatTensor as FT
from torch import LongTensor as LT

# FFNN Parameter
n_hidden = 128
num_classes = 2 # 0 or 1

# sentences
sentences = ["cnn is good for vision","it can classify images well", "object detection techs is matured enough", 
             "deep learning for nlp is growing fast","it can classify words well", "i can build nlp model"]
labels = [1,1,1,0,0,0] # 1 is vision, 0 is nlp

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

inputs = []
for sen in sentences:
    _input = np.zeros([vocab_size])
    _input[[word_dict[n] for n in sen.split()]] = 1
    inputs.append(_input) # ONE-HOT Bag of Words

inputs = FT(inputs)
labels = LT(labels)
    
class FFNN(nn.Module):
    def __init__(self, vocab_size, n_hidden, num_classes):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(vocab_size, n_hidden)
        self.fc2 = nn.Linear(n_hidden, num_classes)
    
    def forward(self, x):
        hidden = self.fc1(x)
        logits = self.fc2(F.relu(hidden))
        return logits
        
        
# Training
model = FFNN(vocab_size, n_hidden, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(inputs)

    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, labels)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()
    
# Test
test_text = ['vision techs is matured', 'deep learning can build nlp model']
tests = []
for text in test_text:
    test = np.zeros([vocab_size])
    test[[word_dict[n] for n in text.split()]] = 1
    tests.append(test)
    
tests = FT(tests)
result = model(tests).data.max(1)[1]

for i, text in enumerate(test_text):
  if result[i] == 0:
      print("\'" + text + "\'", "is about nlp")
  else:
      print("\'"+ text + "\'", "is about vision")

Epoch: 1000 cost = 0.000326
Epoch: 2000 cost = 0.000070
Epoch: 3000 cost = 0.000026
Epoch: 4000 cost = 0.000012
Epoch: 5000 cost = 0.000006
'vision techs is matured' is about vision
'deep learning can build nlp model' is about nlp
